In [1]:
# Install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 199 kB 53.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=82ef32fcf54b728026aa0a07d4abf0fe65efcebbae430d52bc45b5262402423f
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()
# Check Spark Session Information
spark

In [4]:
# Import a Spark function from library
from pyspark.sql.functions import col

In [5]:
# Download Java Virtual Machine (JVM)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-22 22:39:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2022-11-22 22:39:03 (4.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [18]:
from pyspark import SparkFiles

In [19]:
url ="https://devinjaxues-bucket.s3.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)
user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

In [20]:
# Show DataFrame
user_payment_df.show()

+----------+------------+--------------------+
|billing_id|    username|        cc_encrypted|
+----------+------------+--------------------+
|         1|   ibearham0|a799fcafe47d7fb19...|
|         2|    wwaller1|a799fcafe47d7fb19...|
|         3|   ichesnut2|a799fcafe47d7fb19...|
|         4|     tsnarr3|a799fcafe47d7fb19...|
|         5|   fwherrit4|a799fcafe47d7fb19...|
|         6|  fstappard5|a799fcafe47d7fb19...|
|         7|  lhambling6|a799fcafe47d7fb19...|
|         8|      drude7|a799fcafe47d7fb19...|
|         9|   bspawton8|a799fcafe47d7fb19...|
|        10| rmackeller9|a799fcafe47d7fb19...|
|        11| cdennerleya|a799fcafe47d7fb19...|
|        12|    gsarfasb|a799fcafe47d7fb19...|
|        13| mpichefordc|a799fcafe47d7fb19...|
|        14|     bingryd|a799fcafe47d7fb19...|
|        15|   wheinerte|a799fcafe47d7fb19...|
|        16|    mdrewetf|a799fcafe47d7fb19...|
|        17|droughsedgeg|a799fcafe47d7fb19...|
|        18|     abaakeh|a799fcafe47d7fb19...|
|        19| 

In [21]:
url ="https://devinjaxues-bucket.s3.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

In [22]:
# Show DataFrame
user_data_df.show()

+---+----------+---------+-----------+--------------------+--------------------+------------+
| id|first_name|last_name|active_user|      street_address|               state|    username|
+---+----------+---------+-----------+--------------------+--------------------+------------+
|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|   ibearham0|
|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|    wwaller1|
|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|   ichesnut2|
|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|     tsnarr3|
|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|   fwherrit4|
|  6|    Fraser|  Korneev|       true|  76084 Novick Court|           Minnesota|  fstappard5|
|  7|    Demott|   Rapson|       true|    86320 Dahle Park|District of Columbia|  lhambling6|
|  8|    Robert|    Poile|      false|1540 Manitowish Hill| 

In [23]:
#Merged both datasets
joined_df = user_data_df.join(user_payment_df, on="username", how="inner")
joined_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [27]:
#drop null values
dropna_df = joined_df.dropna()
dropna_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [26]:
#load in a sql function to use columns
from pyspark.sql.functions import col

In [28]:
#filter for only colums with active users
cleaned_df = dropna_df.filter(col("active_user") == True)
cleaned_df.show()

+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|     username| id|first_name|  last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|   fstappard5|  6|    Fraser|    Korneev|       true|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|   lhambling6|  7|    Demott|     Rapson|       true|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|    wheinerte| 15|   Sadella|      Jaram|       true|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
| droughsedgeg| 17|    Hewitt|    Trammel|       true|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|    ydudeniei| 19|       Ted|    Knowlys|       true|      31 South Drive|                Ohio| 

In [30]:
#Create active user dataframe
clean_user_df = cleaned_df.select(["id", "first_name", "last_name", "username"])
clean_user_df.show()

+---+----------+-----------+-------------+
| id|first_name|  last_name|     username|
+---+----------+-----------+-------------+
|  6|    Fraser|    Korneev|   fstappard5|
|  7|    Demott|     Rapson|   lhambling6|
| 15|   Sadella|      Jaram|    wheinerte|
| 17|    Hewitt|    Trammel| droughsedgeg|
| 19|       Ted|    Knowlys|    ydudeniei|
| 23|  Annmarie|     Lafond|     fmyttonm|
| 28|      Toma|     Sokell|   bfletcherr|
| 30|       Ram|    Lefever|     gturleyt|
| 31|    Raddie|    Heindle|    calyukinu|
| 33|    Wallie|       Caws| ckleinlererw|
| 34|    Derril|Varfolomeev|  pshanklandx|
| 39|     Kelcy|     Wheway|    enelane12|
| 40|    Dorree|    Rookeby|    sfollet13|
| 41|    Martyn|       Tott|      mtesh14|
| 43|     Cally|      Thody|   tseyfart16|
| 45|       Ted|   Pittaway|   hfarrier18|
| 48|      Fifi|    Lidgley|     nabbie1b|
| 50|    Ashely|     O'Hern|  ystadding1d|
| 53|   Diannne|Osbaldeston|hhallgalley1g|
| 60|     Sonny|     Jeskin|   ageaveny1n|
+---+------

In [31]:
#Create dataframe for billing_info
clean_billing_df = cleaned_df.select(["billing_id", "street_address", "state", "username"])
clean_billing_df.show()

+----------+--------------------+--------------------+-------------+
|billing_id|      street_address|               state|     username|
+----------+--------------------+--------------------+-------------+
|         6|  76084 Novick Court|           Minnesota|   fstappard5|
|         7|    86320 Dahle Park|District of Columbia|   lhambling6|
|        15|7528 Waxwing Terrace|         Connecticut|    wheinerte|
|        17|    2455 Corry Alley|      North Carolina| droughsedgeg|
|        19|      31 South Drive|                Ohio|    ydudeniei|
|        23|     35 Oriole Place|             Georgia|     fmyttonm|
|        28|39641 Eggendart Hill|            Maryland|   bfletcherr|
|        30|   9969 Laurel Alley|               Texas|     gturleyt|
|        31|   811 Talmadge Road|                Ohio|    calyukinu|
|        33|   9999 Kenwood Pass|              Oregon| ckleinlererw|
|        34|     4 Jenifer Court|             Florida|  pshanklandx|
|        39|93207 Morningstar...| 

In [32]:
#Create dataframe for payment_info
clean_payment_df = cleaned_df.select(["billing_id", "cc_encrypted"])
clean_payment_df.show()

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|         6|a799fcafe47d7fb19...|
|         7|a799fcafe47d7fb19...|
|        15|a799fcafe47d7fb19...|
|        17|a799fcafe47d7fb19...|
|        19|a799fcafe47d7fb19...|
|        23|a799fcafe47d7fb19...|
|        28|a799fcafe47d7fb19...|
|        30|a799fcafe47d7fb19...|
|        31|a799fcafe47d7fb19...|
|        33|a799fcafe47d7fb19...|
|        34|a799fcafe47d7fb19...|
|        39|a799fcafe47d7fb19...|
|        40|a799fcafe47d7fb19...|
|        41|a799fcafe47d7fb19...|
|        43|a799fcafe47d7fb19...|
|        45|a799fcafe47d7fb19...|
|        48|a799fcafe47d7fb19...|
|        50|a799fcafe47d7fb19...|
|        53|a799fcafe47d7fb19...|
|        60|a799fcafe47d7fb19...|
+----------+--------------------+
only showing top 20 rows



In [59]:
mode = "append"

In [60]:
jdbc_url="jdbc:postgresql://dataviz.cbs5kmrpfy8t.us-west-2.rds.amazonaws.com:5432/my_data_class_db"

In [58]:
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [56]:
import os

In [63]:
import pandas as pd

In [65]:
from google.colab import files

In [71]:
clean_user_df.write.csv('clean_user_df.csv')

In [72]:
clean_billing_df.write.csv('clean_billing_df.csv')

In [73]:
clean_payment_df.write.csv('clean_payment_df.csv')